In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import zipfile
import shutil

FER_ZIP_PATH = 'fer-2013.zip.zip'
UNZIP_DESTINATION = '/content/extracted_fer_data'
IMG_SIZE = (48, 48)
NUM_CLASSES = 7
BATCH_SIZE = 64
EPOCHS = 30
print("Starting Data Extraction & Preprocessing")

if os.path.exists(UNZIP_DESTINATION):
    shutil.rmtree(UNZIP_DESTINATION)
os.makedirs(UNZIP_DESTINATION, exist_ok=True)
print(f"Destination created: {UNZIP_DESTINATION}")

try:
    print(f"Attempt to extract {FER_ZIP_PATH}...")
    with zipfile.ZipFile(FER_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(UNZIP_DESTINATION)
    print("Extracting successful!!!")
except FileNotFoundError:
    print(f"File Not Found")
    exit()
extracted_contents = os.listdir(UNZIP_DESTINATION)
if len(extracted_contents) == 1 and os.path.isdir(os.path.join(UNZIP_DESTINATION, extracted_contents[0])):
    FER_BASE_DIR = os.path.join(UNZIP_DESTINATION, extracted_contents[0])
else:
    FER_BASE_DIR = UNZIP_DESTINATION

TRAIN_DIR = os.path.join(FER_BASE_DIR, 'train')
TEST_DIR = os.path.join(FER_BASE_DIR, 'test')
VAL_DIR = os.path.join(FER_BASE_DIR, 'validation')

if not os.path.exists(TRAIN_DIR):
    print(f"Training Directory Not Found")
    exit()
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode='grayscale', class_mode='categorical'
)
validation_dir = VAL_DIR if os.path.exists(VAL_DIR) else TEST_DIR

validation_generator = test_datagen.flow_from_directory(
    validation_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode='grayscale', class_mode='categorical'
)
print("Preprocessing Process Completed Successfully!!!")
print("Finally Milestone-1 Completed")

model = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])


callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Model Summary ---")
model.summary()
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)
MODEL_SAVE_PATH = 'emotion_detection_cnn_model.keras'
model.save(MODEL_SAVE_PATH)
print(f"\n Milestone 2 Output Saved: Model saved to {MODEL_SAVE_PATH}")
loaded_model = tf.keras.models.load_model('emotion_detection_cnn_model.keras')
print(" Milestone 2 Completed!!!")

Starting Data Extraction & Preprocessing
Destination created: /content/extracted_fer_data
Attempt to extract fer-2013.zip.zip...
Extracting successful!!!
Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Preprocessing Process Completed Successfully!!!
Finally Milestone-1 Completed


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



--- Model Summary ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,79

...

 Non-trainable params: 896 (3.50 KB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 636s 1s/step - accuracy: 0.2287 - loss: 2.1715 - val_accuracy: 0.2687 - val_loss: 1.7765 - learning_rate: 0.0010
Epoch 2/30
  1/448 ━━━━━━━━━━━━━━━━━━━━ 9:13 1s/step - accuracy: 0.2812 - loss: 1.7434

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


448/448 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.2812 - loss: 1.7434 - val_accuracy: 0.2681 - val_loss: 1.7787 - learning_rate: 0.0010
Epoch 3/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.3003 - loss: 1.7198 - val_accuracy: 0.4177 - val_loss: 1.4752 - learning_rate: 0.0010
Epoch 4/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.4219 - loss: 1.4719 - val_accuracy: 0.4107 - val_loss: 1.4848 - learning_rate: 0.0010
Epoch 5/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 681s 1s/step - accuracy: 0.3508 - loss: 1.6203 - val_accuracy: 0.3884 - val_loss: 1.5456 - learning_rate: 0.0010
Epoch 6/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.3438 - loss: 1.7285 - val_accuracy: 0.3878 - val_loss: 1.5484 - learning_rate: 0.0010
Epoch 7/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 680s 2s/step - accuracy: 0.3777 - loss: 1.5493 - val_accuracy: 0.4612 - val_loss: 1.3563 - learning_rate: 0.0010
Epoch 8/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.4531 - loss: 1.4086 - 

In [12]:
import numpy as np
import pandas as pd
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.models import load_model

MUSIC_ZIP_PATH = 'spotify_dataset.zip'
MUSIC_CSV_IN_ZIP = 'spotify_tracks.csv'
OUTPUT_DIR = 'preprocessed_data'
MODEL_SAVE_PATH ='emotion_detection_cnn_model.keras'
MUSIC_DATA_PATH = os.path.join(OUTPUT_DIR, 'preprocessed_music_data.csv')


EMOTION_TO_MOOD_MAP = {
    'happy': ['pop', 'dance', 'happy', 'party', 'upbeat', 'electronic'],
    'sad': ['acoustic', 'blues', 'sad', 'lo-fi', 'gospel'],
    'neutral': ['ambient', 'chill', 'study', 'sleep', 'jazz'],
    'angry': ['rock', 'metal', 'hardcore', 'grindcore', 'punk'],
    'fear': ['ambient', 'soundtracks', 'movies', 'drone'],
    'surprise': ['dance', 'j-dance', 'club', 'techno'],
    'disgust': ['industrial', 'metal-misc', 'goth']
}
EMOTION_LABELS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def preprocess_music_data():
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    print("Starting Music Data Preprocessing from Milestone 1....")
    try:
        with zipfile.ZipFile(MUSIC_ZIP_PATH, 'r') as z:
            with z.open(MUSIC_CSV_IN_ZIP) as f:
                df = pd.read_csv(f)

    except FileNotFoundError:

        print(f"File not found at: {MUSIC_ZIP_PATH}.")
        return False
    except Exception as e:
        print(f"ERROR at reading music CSV file: {e}")
        return False

    df = df[['name', 'artists', 'genre', 'popularity']].dropna().drop_duplicates()

    def get_mood_tags(genre):
        genre_lower = str(genre).lower()
        all_tags = [tag for tags in EMOTION_TO_MOOD_MAP.values() for tag in tags]
        return [tag for tag in all_tags if tag in genre_lower]

    df['Mood_Tags'] = df['genre'].apply(get_mood_tags)
    df.to_csv(MUSIC_DATA_PATH, index=False)
    print(f"Music Data Preprocessing Completed successfully!!!.File saved to: {MUSIC_DATA_PATH}")
    return True

def recommend_music(emotion_label, music_df, top_n=10):
    if emotion_label not in EMOTION_TO_MOOD_MAP:
        return pd.DataFrame()

    required_tags = EMOTION_TO_MOOD_MAP[emotion_label.lower()]

    print(f"\nRecommendation for: {emotion_label.upper()}")
    print(f"Searching for tracks matching tags: {required_tags}")
    def check_for_tags(tags_str):
        if pd.isna(tags_str) or tags_str == '[]' or not tags_str: return False
        tags_str_lower = str(tags_str).lower()
        return any(tag in tags_str_lower for tag in required_tags)

    filtered_df = music_df[music_df['Mood_Tags'].apply(check_for_tags)].copy()

    if filtered_df.empty:
        print("No exact matching tracks found")
        return music_df.sort_values(by='popularity', ascending=False).head(top_n)[['name', 'artists', 'genre', 'popularity']]

    recommendations = filtered_df.sort_values(by='popularity', ascending=False).head(top_n)

    return recommendations[['name', 'artists', 'genre', 'popularity']]

def run_moodmate_demo(simulated_emotion='happy'):
    print(f"\nEmotion Detected from Milestone 2 Output: {simulated_emotion.capitalize()}")

    try:
        music_df = pd.read_csv(MUSIC_DATA_PATH)
        if os.path.exists(MODEL_SAVE_PATH):
            print("Model and Music Data files verified")
        else:
            print(f"File {MODEL_SAVE_PATH} not found")

    except Exception as e:
        print(f"Failed to load music data...Details: {e}")
        return

    detected_emotion = simulated_emotion
    recommendations = recommend_music(detected_emotion, music_df)

    print("\nMusic Recommendation Generated Successfully!!")
    with pd.option_context('display.max_rows', None,
                           'display.max_columns', None,
                           'display.width', 1000):

        table_output = recommendations.to_string(index=False)

        print("\n=======================================================")
        print(f"       FINAL RECOMMENDATION for {detected_emotion.upper()}")
        print("=======================================================")
        print(table_output)
        print("=======================================================")


if __name__ == "__main__":

    print("\n Starting Music Recommendation Engine")

    run_moodmate_demo(simulated_emotion='angry')
    run_moodmate_demo(simulated_emotion='sad')

    print("\nMilestone 3 Completed Successfully!!!")


 Starting Music Recommendation Engine

Emotion Detected from Milestone 2 Output: Angry
Model and Music Data files verified

Recommendation for: ANGRY
Searching for tracks matching tags: ['rock', 'metal', 'hardcore', 'grindcore', 'punk']

Music Recommendation Generated Successfully!!

       FINAL RECOMMENDATION for ANGRY
                                                   name                     artists       genre  popularity
                                             Pink Skies                  Zach Bryan        punk          85
                                                Starboy       The Weeknd, Daft Punk        punk          84
                                               ROCKSTAR                    Junior H        rock          78
                                             West Coast                Lana Del Rey rock-n-roll          78
              Praise The Lord (Da Shine) (feat. Skepta)          A$AP Rocky, Skepta        rock          78
                            